In [16]:
import openai
from openai import OpenAI
import json



client = OpenAI(api_key='')



# Function to generate question-answer pairs for a batch of paragraphs
def generate_qa_pairs(paragraphs):
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON. Generate a list of question-answer pairs for the paragraph fields in the for the json array provided by the user. Format the output as JSON with 'question', 'answer', and 'context' fields."},
            {"role": "user", "content": f"{json.dumps(paragraphs)}"}
        ]
    )
    
    return response


def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [23]:
with open('paragraphs.json', 'r') as file:
    paragraphs = json.load(file)

    chunks = list(divide_chunks(paragraphs, 10))
    
    chunk_number = 0
    for chunk in chunks:
        response = generate_qa_pairs(chunk)
        with open(f'synthetic-datasets/qac-paris-chunk{chunk_number}.json', 'w', encoding="utf-8") as file:
            json.dump(response.choices[0].message.content, file, indent=2)
            print(f'Wrote chunk {chunk_number}: synthetic-datasets/qac-paris-chunk{chunk_number}')
        chunk_number += 1
    


Wrote chunk 0: synthetic-datasets/qac-paris-chunk0
Wrote chunk 1: synthetic-datasets/qac-paris-chunk1
Wrote chunk 2: synthetic-datasets/qac-paris-chunk2
Wrote chunk 3: synthetic-datasets/qac-paris-chunk3
Wrote chunk 4: synthetic-datasets/qac-paris-chunk4
Wrote chunk 5: synthetic-datasets/qac-paris-chunk5
Wrote chunk 6: synthetic-datasets/qac-paris-chunk6
Wrote chunk 7: synthetic-datasets/qac-paris-chunk7
Wrote chunk 8: synthetic-datasets/qac-paris-chunk8
Wrote chunk 9: synthetic-datasets/qac-paris-chunk9
Wrote chunk 10: synthetic-datasets/qac-paris-chunk10
Wrote chunk 11: synthetic-datasets/qac-paris-chunk11
Wrote chunk 12: synthetic-datasets/qac-paris-chunk12
Wrote chunk 13: synthetic-datasets/qac-paris-chunk13
Wrote chunk 14: synthetic-datasets/qac-paris-chunk14
Wrote chunk 15: synthetic-datasets/qac-paris-chunk15
Wrote chunk 16: synthetic-datasets/qac-paris-chunk16
Wrote chunk 17: synthetic-datasets/qac-paris-chunk17


In [15]:
print(response.choices[0].message.content)


  {
    "question": "What is the main content covered in the Zulassungssatzung for the Masterstudiengänge at Hochschule Konstanz?",
    "answer": "The Zulassungssatzung covers various sections including the application area, deadlines, general provisions, specific master programs, and final provisions.",
    "context": "Hochschule Konstanz Technik, Wirtschaft und Gestaltung Seite 1 von 43 Zulassungssatzung für die Masterstudiengänge (ZuSMa) (vom 09. Februar 2021, zuletzt geändert am 18. Januar 2022) Inhaltsverzeichnis Teil 1 – Anwendungsbereich ..."
  }
  
  


In [35]:
import os
import json

def merge_json_files(directory):
    merged_data = []
    
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = directory + filename
            with open(filepath, "r", encoding="utf-8") as file:
                data = json.loads(json.load(file))
                first = list(data)[0]
                print(first)
                #print(data)
                merged_data.extend(data[first])
    
    print(f"Dataset Length: {len(merged_data)}")
    with open("synthetic-datasets/synthetic-qac-pairs.json", "w", encoding="utf-8") as outfile:
        json.dump(merged_data, outfile, indent=2)

directory_path = "synthetic-datasets/"
merge_json_files(directory_path)

questionAnswerPairs
questions
question_answers
question_answer_pairs
QA_pairs
questionAnswerPairs


TypeError: the JSON object must be str, bytes or bytearray, not list

Dataset Length: 240


In [39]:
from sklearn.model_selection import train_test_split
import os 
import json

with open("synthetic-datasets/synthetic-qac-pairs.json", "r", encoding="utf-8") as file:
        data = json.load(file)
        print(f"Dataset Length: {len(data)}")

        finetune_topic = "chatbot-qac-pairs"

        output_path_train = f"datasets/{finetune_topic}/train.json"
        output_path_validation = f"datasets/{finetune_topic}/validation.json"

        train_data, validation_data = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=True)
        print(f"LENGTH TRAINING: {len(train_data)}")
        print(f"LENGTH VALIDATION: {len(validation_data)}")
        print(f"LENGTH TOTAL: {len(data)}")

        os.makedirs(os.path.dirname(output_path_train), exist_ok=True)
        with open(output_path_train, "w", encoding="utf-8") as output_file:
            json.dump(train_data, output_file, indent=2)

        os.makedirs(os.path.dirname(output_path_validation), exist_ok=True)
        with open(output_path_validation, "w", encoding="utf-8") as output_file:
            json.dump(validation_data, output_file, indent=2)

Dataset Length: 240
LENGTH TRAINING: 192
LENGTH VALIDATION: 48
LENGTH TOTAL: 240
